## Thư viện

### Cài thư viện

### Import thư viện

In [ ]:
import re

## Dữ liệu

### Tải dữ liệu

In [ ]:
!wget -O "/content/data.zip" "https://drive.google.com/uc?export=download&id=1lXXGI4aY1NVzwvz3NLHFAvhwk9LBnDMB" 

--2023-04-19 15:50:01--  https://drive.google.com/uc?export=download&id=1lXXGI4aY1NVzwvz3NLHFAvhwk9LBnDMB
Resolving drive.google.com (drive.google.com)... 142.251.162.101, 142.251.162.102, 142.251.162.113, ...
Connecting to drive.google.com (drive.google.com)|142.251.162.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-14-3s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/0fhmrv0h1bral33reevlk5roc3offnum/1681919400000/02702851125710553677/*/1lXXGI4aY1NVzwvz3NLHFAvhwk9LBnDMB?e=download&uuid=79f790fa-22f8-46da-8e3d-e54dca377132 [following]
--2023-04-19 15:50:03--  https://doc-14-3s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/0fhmrv0h1bral33reevlk5roc3offnum/1681919400000/02702851125710553677/*/1lXXGI4aY1NVzwvz3NLHFAvhwk9LBnDMB?e=download&uuid=79f790fa-22f8-46da-8e3d-e54dca377132
Resolving doc-14-3s-docs.googleusercontent.com (doc-14-3s-docs.googleusercontent.com)... 173.194.215.132, 

In [ ]:
!unzip "data.zip"

Archive:  data.zip
replace SA/Guidelines-SA-Hotel (4-3-2018).pdf? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: SA/Guidelines-SA-Hotel (4-3-2018).pdf  
  inflating: SA/Guidelines-SA-Restaurant (5-3-2018).pdf  
  inflating: SA/VLSP2018-SA-train-dev-test.zip  


In [ ]:
!unzip "/content/SA/VLSP2018-SA-train-dev-test.zip"

Archive:  /content/SA/VLSP2018-SA-train-dev-test.zip
replace VLSP2018-SA-train-dev-test/3-VLSP2018-SA-Restaurant-test (8-3-2018).txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: VLSP2018-SA-train-dev-test/3-VLSP2018-SA-Restaurant-test (8-3-2018).txt  
  inflating: VLSP2018-SA-train-dev-test/1-VLSP2018-SA-Hotel-train (7-3-2018).txt  
  inflating: VLSP2018-SA-train-dev-test/3-VLSP2018-SA-Hotel-test (8-3-2018).txt  
  inflating: VLSP2018-SA-train-dev-test/1-VLSP2018-SA-Restaurant-train (7-3-2018).txt  
  inflating: VLSP2018-SA-train-dev-test/2-VLSP2018-SA-Hotel-dev (7-3-2018).txt  
  inflating: VLSP2018-SA-train-dev-test/2-VLSP2018-SA-Restaurant-dev (7-3-2018).txt  


### Xử lí dữ liệu

In [ ]:
VNalpha = 'àáãạảăắằẳẵặâấầẩẫậèéẹẻẽêềếểễệđìíĩỉịòóõọỏôốồổỗộơớờởỡợùúũụủưứừửữựỳỵỷỹýÀÁÃẠẢĂẮẰẲẴẶÂẤẦẨẪẬÈÉẸẺẼÊỀẾỂỄỆĐÌÍĨỈỊÒÓÕỌỎÔỐỒỔỖỘƠỚỜỞỠỢÙÚŨỤỦƯỨỪỬỮỰỲỴỶỸÝ'

In [ ]:
def preprocess_txt(file_link):
    '''
    * Xử lí file txt chứa đường dẫn file txt thành một mảng chứa các điểm dữ liệu.
    * Một điểm dữ liệu là tuple (x, y) với:  
      + x: Mảng chứa các từ, một dấu câu xem là một tiếng (giữ dấu câu để xét trong việc dùng n-grams về sau).
      + y: Mảng các tuple (y1, y2). y1 là Aspect, y2 là Sentiment.
    '''
    '''
    Dữ liệu trong file txt có sự lặp lại cấu trúc mỗi 4 dòng với kết cấu sau:
    #1                                               -> chỉ số
    Khách sạn đẹp                                    -> câu
    {Aspect1, Sentiment1}, {Aspect2, Sentiment2} ... -> aspect và sentiment
                                                     -> dòng trống  
    '''
    with open(file_link, encoding='utf8') as f:
        lines = f.readlines()
        result = []
        temp_str = ""
        for line in lines:
            if line == "" or line[0] == "#":  # Không quan tâm dòng trống hay dòng chỉ số
                continue
            if line[0] == "{":                # Dòng aspect và sentiment
                line = re.findall(r'{[^{}]+}', line)
                line = [tuple(txt[1:-1].replace(' ', '').split(',')) for txt in line]
                result.append((temp_str, line))
            if line[0] != "{":                # Dòng text
                temp_str = str(line).rstrip()
    return result
            

In [ ]:
preprocess_txt('/content/VLSP2018-SA-train-dev-test/1-VLSP2018-SA-Hotel-train (7-3-2018).txt')[:3]

[('Rộng rãi KS mới nhưng rất vắng. Các dịch vụ chất lượng chưa cao và thiếu.',
  [('HOTEL#DESIGN&FEATURES', 'positive'), ('HOTEL#GENERAL', 'negative')]),
 ('Địa điểm thuận tiện, trong vòng bán kính 1,5km nhiều quán ăn ngon',
  [('LOCATION#GENERAL', 'positive')]),
 ('Phục vụ, view đẹp, vị trí',
  [('SERVICE#GENERAL', 'positive'),
   ('HOTEL#GENERAL', 'positive'),
   ('LOCATION#GENERAL', 'positive')])]

In [ ]:
def get_aspect_list(data):
    '''
    Lấy danh sách aspect từ dữ liệu.
    '''
    result = []
    for text, label_lst in data:
        for asp, sent in label_lst:
            result.append(asp)
    return list(set(result))
def get_sentiment_list(data):
    '''
    Lấy danh sách sentiment từ dữ liệu.
    '''
    result = []
    for text, label_lst in data:
        for asp, sent in label_lst:
            result.append(sent)
    return list(set(result))
def process_label(data):
    '''
    Gom nhóm dữ liệu theo label.
    Đầu vào: Dữ liệu theo chuẩn đầu ra của hàm preprocess_txt.
    Đầu ra: Dữ liệu kiểu dict:
      + key là aspect.
      + value là một mảng các tuple (x, y). x là text, y là sentiment.
    sentiment được đánh số:
      1: negative
      2: positive
      3: neutral
      0: not mentioned
    '''
    sent_list = get_sentiment_list(data)
    asp_list = get_aspect_list(data)
    asp_map = {'negative' : 1, 'positive' : 2, 'neutral' : 3}
    result = {}
    for asp in asp_list:
        result[asp] = {}
    for text, label_lst in data:
        for asp in asp_list:
            result[asp][text] = 0
        for asp, sent in label_lst:
            result[asp][text] = asp_map[sent]
    return result

In [ ]:
def process_raw_dataset(text):
    '''
    Xử lí các vấn đề sau:
    - Viết hoa thường lẫn lộn
    - Viết dấu, không dấu lẫn lộn
    - Xoá số, dấu câu, kí tự đặc biệt
    '''
    text = text.lower()
    VNalpha = 'àáãạảăắằẳẵặâấầẩẫậèéẹẻẽêềếểễệđìíĩỉịòóõọỏôốồổỗộơớờởỡợùúũụủưứừửữựỳỵỷỹýÀÁÃẠẢĂẮẰẲẴẶÂẤẦẨẪẬÈÉẸẺẼÊỀẾỂỄỆĐÌÍĨỈỊÒÓÕỌỎÔỐỒỔỖỘƠỚỜỞỠỢÙÚŨỤỦƯỨỪỬỮỰỲỴỶỸÝ'
    VNkodau = 'aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyaaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyy'
    for idx in range(len(VNalpha)):
        text = text.replace(VNalpha[idx], VNkodau[idx])
    text = re.sub(r'[^(a-zA-Z )]+', '', text)
    return text

In [ ]:
process_raw_dataset("Hôm nay, tôi ĐI HỌC!")

'hom nay toi di hoc'

## Mô hình

### Hotel

#### Huấn luyện mô hình

In [ ]:
# Xử lí dữ liệu theo các hàm phía trên
processed_txt_data = preprocess_txt('/content/VLSP2018-SA-train-dev-test/1-VLSP2018-SA-Hotel-train (7-3-2018).txt')
processed_txt_data = [(process_raw_dataset(text), label) for text, label in processed_txt_data]
data = process_label(processed_txt_data)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
import pandas as pd

In [ ]:
# dựng corpus (chứa tất cả các text), dựng mô hình để tạo BoW
corpus = [text for txt in list(data.values()) for text in list(txt.keys())]
vectorizer = CountVectorizer()
vectorizer.fit_transform(corpus)

<97119x2775 sparse matrix of type '<class 'numpy.int64'>'
	with 4219347 stored elements in Compressed Sparse Row format>

In [ ]:
# Tạo mô hình
# Mỗi aspect có 1 mô hình riêng, đầu vào là text, đầu ra là 0, 1, 2, hoăc 3 tương ứng với not mentioned, negative, positive, neutral
asp_lst = get_aspect_list(processed_txt_data)
model = {}
for asp in asp_lst:
    model[asp] = LogisticRegression(verbose=0, solver='liblinear',random_state=0, C=5, penalty='l2',max_iter=100)
    x = [txt for txt, label in data[asp].items()]
    y = [label for txt, label in data[asp].items()]
    x = vectorizer.transform(x)
    model[asp].fit(x.toarray(), y)

#### Thử nghiệm trên tập val và test

In [ ]:
def predict(text, model, vectorizer, asp_lst):
    '''
    Lấy kết quả dự đoán từ model tương ứng với text, model là một dict các naive bayes tương ứng với từng aspect.
    '''
    result = []
    asp_map = {1: 'negative', 2: 'positive', 3: 'neutral'}
    for asp in asp_lst:
        pred = model[asp].predict([vectorizer.transform([text]).toarray()[0]])[0]
        if pred != 0:
            result.append((asp, asp_map[pred]))
    return result

In [ ]:
def check_pred(pred, truth):
    '''
    Trả về TP so giữa pred và truth.
    '''
    TP = len(list(set(pred) & set(truth)))
    return TP
def check_pred_asp(pred, truth):
    '''
    Trả về TP so giữa pred và truth, chỉ xét aspect.
    '''
    pred = [asp for asp, sent in pred]
    truth = [asp for asp, sent in truth]
    TP = len(list(set(pred) & set(truth)))
    return TP

In [ ]:
def F1(TP, pred, truth):
    '''
    Trả về chỉ số F1. 
    '''
    precision = TP / pred
    recall = TP / truth
    return 2 * precision * recall / (precision + recall)

In [ ]:
# dùng chỉ số F1 để đánh giá
val_data = preprocess_txt('/content/VLSP2018-SA-train-dev-test/2-VLSP2018-SA-Hotel-dev (7-3-2018).txt')
nTP, nTP_asp, npred, ntruth = 0, 0, 0, 0 
asp_f1 = {}
sent_f1 = {}
for pdata in val_data:
    pdata = (process_raw_dataset(pdata[0]), pdata[1])
    pred = predict(pdata[0], model, vectorizer, asp_lst)
    truth = pdata[1]

    nTP = nTP + check_pred(pred, truth)
    nTP_asp = nTP_asp + check_pred_asp(pred, truth)
    npred = npred + len(pred)
    ntruth = ntruth + len(truth)
    ints = set(pred) & set(truth)
    for m in ints:
        if m[0] not in asp_f1.keys():
            asp_f1[m[0]] = (0, 0, 0)
        if m[1] not in sent_f1.keys():
            sent_f1[m[1]] = (0, 0, 0)
        asp_f1[m[0]] = (asp_f1[m[0]][0] + 1, asp_f1[m[0]][1], asp_f1[m[0]][2])
        sent_f1[m[1]] = (sent_f1[m[1]][0] + 1, sent_f1[m[1]][1], sent_f1[m[1]][2])
    for m in set(pred) - ints:
        if m[0] not in asp_f1.keys():
            asp_f1[m[0]] = (0, 0, 0)
        if m[1] not in sent_f1.keys():
            sent_f1[m[1]] = (0, 0, 0)
        asp_f1[m[0]] = ((asp_f1[m[0]])[0], (asp_f1[m[0]])[1] + 1, (asp_f1[m[0]])[2])
        sent_f1[m[1]] = (sent_f1[m[1]][0], sent_f1[m[1]][1] + 1, sent_f1[m[1]][2])
    for m in set(truth) - ints:
        if m[0] not in asp_f1.keys():
            asp_f1[m[0]] = (0, 0, 0)
        if m[1] not in sent_f1.keys():
            sent_f1[m[1]] = (0, 0, 0)
        asp_f1[m[0]] = (asp_f1[m[0]][0], asp_f1[m[0]][1], asp_f1[m[0]][2] + 1)
        sent_f1[m[1]] = (sent_f1[m[1]][0], sent_f1[m[1]][1], sent_f1[m[1]][2] + 1)
print('TẬP TEST')
report = pd.DataFrame.from_dict(asp_f1, orient='index', columns=['TP', 'FP', 'FN'])
report['Precision'] = report['TP'] / (report['TP'] + report['FP'])
report['Recall'] = report['TP'] / (report['TP'] + report['FN'])
report['F1'] = 2 * (report['Precision'] * report['Recall']) / (report['Precision'] + report['Recall'])
report.drop(['TP', 'FN', 'FP'], axis=1, inplace=True)
print(report)
report = pd.DataFrame.from_dict(sent_f1, orient='index', columns=['TP', 'FP', 'FN'])
report['Precision'] = report['TP'] / (report['TP'] + report['FP'])
report['Recall'] = report['TP'] / (report['TP'] + report['FN'])
report['F1'] = 2 * (report['Precision'] * report['Recall']) / (report['Precision'] + report['Recall'])
report.drop(['TP', 'FN', 'FP'], axis=1, inplace=True)
print(report)
print('F1:', F1(nTP, npred, ntruth))

TẬP TEST
                                Precision    Recall        F1
LOCATION#GENERAL                 0.809612  0.772487  0.790614
ROOMS#DESIGN&FEATURES            0.502717  0.389474  0.438909
SERVICE#GENERAL                  0.867412  0.828375  0.847444
FOOD&DRINKS#QUALITY              0.764706  0.614657  0.681520
ROOM_AMENITIES#COMFORT           0.719101  0.263374  0.385542
HOTEL#CLEANLINESS                0.525424  0.558559  0.541485
FACILITIES#DESIGN&FEATURES       0.378641  0.177273  0.241486
HOTEL#COMFORT                    0.520833  0.425791  0.468541
ROOM_AMENITIES#CLEANLINESS       0.266667  0.075472  0.117647
HOTEL#GENERAL                    0.648352  0.559242  0.600509
ROOM_AMENITIES#QUALITY           0.320000  0.196721  0.243655
HOTEL#DESIGN&FEATURES            0.363208  0.283088  0.318182
FOOD&DRINKS#STYLE&OPTIONS        0.637555  0.505190  0.563707
ROOMS#COMFORT                    0.413043  0.268551  0.325482
ROOM_AMENITIES#DESIGN&FEATURES   0.460123  0.179856  0.258621

In [ ]:
val_data = preprocess_txt('/content/VLSP2018-SA-train-dev-test/3-VLSP2018-SA-Hotel-test (8-3-2018).txt')
nTP, nTP_asp, npred, ntruth = 0, 0, 0, 0 
asp_f1 = {}
sent_f1 = {}
for pdata in val_data:
    pdata = (process_raw_dataset(pdata[0]), pdata[1])
    pred = predict(pdata[0], model, vectorizer, asp_lst)
    truth = pdata[1]

    nTP = nTP + check_pred(pred, truth)
    nTP_asp = nTP_asp + check_pred_asp(pred, truth)
    npred = npred + len(pred)
    ntruth = ntruth + len(truth)
    ints = set(pred) & set(truth)
    for m in ints:
        if m[0] not in asp_f1.keys():
            asp_f1[m[0]] = (0, 0, 0)
        if m[1] not in sent_f1.keys():
            sent_f1[m[1]] = (0, 0, 0)
        asp_f1[m[0]] = (asp_f1[m[0]][0] + 1, asp_f1[m[0]][1], asp_f1[m[0]][2])
        sent_f1[m[1]] = (sent_f1[m[1]][0] + 1, sent_f1[m[1]][1], sent_f1[m[1]][2])
    for m in set(pred) - ints:
        if m[0] not in asp_f1.keys():
            asp_f1[m[0]] = (0, 0, 0)
        if m[1] not in sent_f1.keys():
            sent_f1[m[1]] = (0, 0, 0)
        asp_f1[m[0]] = ((asp_f1[m[0]])[0], (asp_f1[m[0]])[1] + 1, (asp_f1[m[0]])[2])
        sent_f1[m[1]] = (sent_f1[m[1]][0], sent_f1[m[1]][1] + 1, sent_f1[m[1]][2])
    for m in set(truth) - ints:
        if m[0] not in asp_f1.keys():
            asp_f1[m[0]] = (0, 0, 0)
        if m[1] not in sent_f1.keys():
            sent_f1[m[1]] = (0, 0, 0)
        asp_f1[m[0]] = (asp_f1[m[0]][0], asp_f1[m[0]][1], asp_f1[m[0]][2] + 1)
        sent_f1[m[1]] = (sent_f1[m[1]][0], sent_f1[m[1]][1], sent_f1[m[1]][2] + 1)
print('TẬP TEST')
report = pd.DataFrame.from_dict(asp_f1, orient='index', columns=['TP', 'FP', 'FN'])
report['Precision'] = report['TP'] / (report['TP'] + report['FP'])
report['Recall'] = report['TP'] / (report['TP'] + report['FN'])
report['F1'] = 2 * (report['Precision'] * report['Recall']) / (report['Precision'] + report['Recall'])
report.drop(['TP', 'FN', 'FP'], axis=1, inplace=True)
print(report)
report = pd.DataFrame.from_dict(sent_f1, orient='index', columns=['TP', 'FP', 'FN'])
report['Precision'] = report['TP'] / (report['TP'] + report['FP'])
report['Recall'] = report['TP'] / (report['TP'] + report['FN'])
report['F1'] = 2 * (report['Precision'] * report['Recall']) / (report['Precision'] + report['Recall'])
report.drop(['TP', 'FN', 'FP'], axis=1, inplace=True)
print(report)
print('F1:', F1(nTP, npred, ntruth))

TẬP TEST
                                Precision    Recall        F1
SERVICE#GENERAL                  0.830882  0.814904  0.822816
ROOM_AMENITIES#CLEANLINESS       0.529412  0.195652  0.285714
LOCATION#GENERAL                 0.817308  0.769231  0.792541
ROOMS#CLEANLINESS                0.661538  0.645000  0.653165
ROOMS#COMFORT                    0.405405  0.322581  0.359281
ROOMS#GENERAL                    0.439024  0.315789  0.367347
HOTEL#GENERAL                    0.649254  0.576159  0.610526
ROOMS#DESIGN&FEATURES            0.503597  0.353535  0.415430
ROOM_AMENITIES#DESIGN&FEATURES   0.528571  0.256944  0.345794
HOTEL#CLEANLINESS                0.370787  0.492537  0.423077
ROOM_AMENITIES#COMFORT           0.566667  0.250000  0.346939
ROOM_AMENITIES#GENERAL           0.419355  0.406250  0.412698
ROOM_AMENITIES#QUALITY           0.317073  0.220339  0.260000
FOOD&DRINKS#STYLE&OPTIONS        0.610000  0.491935  0.544643
ROOMS#QUALITY                    0.125000  0.100000  0.111111

### Restaurant

#### Huấn luyện mô hình

In [ ]:
processed_txt_data = preprocess_txt('/content/VLSP2018-SA-train-dev-test/1-VLSP2018-SA-Restaurant-train (7-3-2018).txt')
processed_txt_data = [(process_raw_dataset(text), label) for text, label in processed_txt_data]
data = process_label(processed_txt_data)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [ ]:
corpus = [text for txt in list(data.values()) for text in list(txt.keys())]
vectorizer = CountVectorizer()
vectorizer.fit_transform(corpus)

<34980x3822 sparse matrix of type '<class 'numpy.int64'>'
	with 1429968 stored elements in Compressed Sparse Row format>

In [ ]:
asp_lst = get_aspect_list(processed_txt_data)
model = {}
for asp in asp_lst:
    model[asp] = LogisticRegression(verbose=0, solver='liblinear',random_state=0, C=5, penalty='l2',max_iter=100)
    x = [txt for txt, label in data[asp].items()]
    y = [label for txt, label in data[asp].items()]
    x = vectorizer.transform(x)
    model[asp].fit(x.toarray(), y)

#### Thử nghiệm trên tập val và test

In [ ]:
def predict(text, model, vectorizer, asp_lst):
    '''
    Lấy kết quả dự đoán từ model tương ứng với text, model là một dict các naive bayes tương ứng với từng aspect.
    '''
    result = []
    asp_map = {1: 'negative', 2: 'positive', 3: 'neutral'}
    for asp in asp_lst:
        pred = model[asp].predict([vectorizer.transform([text]).toarray()[0]])[0]
        if pred != 0:
            result.append((asp, asp_map[pred]))
    return result

In [ ]:
def check_pred(pred, truth):
    '''
    Trả về TP so giữa pred và truth.
    '''
    TP = len(list(set(pred) & set(truth)))
    return TP
def check_pred_asp(pred, truth):
    '''
    Trả về TP so giữa pred và truth, chỉ xét aspect.
    '''
    pred = [asp for asp, sent in pred]
    truth = [asp for asp, sent in truth]
    TP = len(list(set(pred) & set(truth)))
    return TP

In [ ]:
def F1(TP, pred, truth):
    '''
    Trả về chỉ số F1. 
    '''
    precision = TP / pred
    recall = TP / truth
    return 2 * precision * recall / (precision + recall)

In [ ]:
val_data = preprocess_txt('/content/VLSP2018-SA-train-dev-test/2-VLSP2018-SA-Restaurant-dev (7-3-2018).txt')
nTP, nTP_asp, npred, ntruth = 0, 0, 0, 0 
asp_f1 = {}
sent_f1 = {}
for pdata in val_data:
    pdata = (process_raw_dataset(pdata[0]), pdata[1])
    pred = predict(pdata[0], model, vectorizer, asp_lst)
    truth = pdata[1]

    nTP = nTP + check_pred(pred, truth)
    nTP_asp = nTP_asp + check_pred_asp(pred, truth)
    npred = npred + len(pred)
    ntruth = ntruth + len(truth)
    ints = set(pred) & set(truth)
    for m in ints:
        if m[0] not in asp_f1.keys():
            asp_f1[m[0]] = (0, 0, 0)
        if m[1] not in sent_f1.keys():
            sent_f1[m[1]] = (0, 0, 0)
        asp_f1[m[0]] = (asp_f1[m[0]][0] + 1, asp_f1[m[0]][1], asp_f1[m[0]][2])
        sent_f1[m[1]] = (sent_f1[m[1]][0] + 1, sent_f1[m[1]][1], sent_f1[m[1]][2])
    for m in set(pred) - ints:
        if m[0] not in asp_f1.keys():
            asp_f1[m[0]] = (0, 0, 0)
        if m[1] not in sent_f1.keys():
            sent_f1[m[1]] = (0, 0, 0)
        asp_f1[m[0]] = ((asp_f1[m[0]])[0], (asp_f1[m[0]])[1] + 1, (asp_f1[m[0]])[2])
        sent_f1[m[1]] = (sent_f1[m[1]][0], sent_f1[m[1]][1] + 1, sent_f1[m[1]][2])
    for m in set(truth) - ints:
        if m[0] not in asp_f1.keys():
            asp_f1[m[0]] = (0, 0, 0)
        if m[1] not in sent_f1.keys():
            sent_f1[m[1]] = (0, 0, 0)
        asp_f1[m[0]] = (asp_f1[m[0]][0], asp_f1[m[0]][1], asp_f1[m[0]][2] + 1)
        sent_f1[m[1]] = (sent_f1[m[1]][0], sent_f1[m[1]][1], sent_f1[m[1]][2] + 1)
print('TẬP TEST')
report = pd.DataFrame.from_dict(asp_f1, orient='index', columns=['TP', 'FP', 'FN'])
report['Precision'] = report['TP'] / (report['TP'] + report['FP'])
report['Recall'] = report['TP'] / (report['TP'] + report['FN'])
report['F1'] = 2 * (report['Precision'] * report['Recall']) / (report['Precision'] + report['Recall'])
report.drop(['TP', 'FN', 'FP'], axis=1, inplace=True)
print(report)
report = pd.DataFrame.from_dict(sent_f1, orient='index', columns=['TP', 'FP', 'FN'])
report['Precision'] = report['TP'] / (report['TP'] + report['FP'])
report['Recall'] = report['TP'] / (report['TP'] + report['FN'])
report['F1'] = 2 * (report['Precision'] * report['Recall']) / (report['Precision'] + report['Recall'])
report.drop(['TP', 'FN', 'FP'], axis=1, inplace=True)
print(report)
print('F1:', F1(nTP, npred, ntruth))

TẬP TEST
                          Precision    Recall        F1
FOOD#QUALITY               0.812950  0.779310  0.795775
FOOD#STYLE&OPTIONS         0.504438  0.660853  0.572148
FOOD#PRICES                0.543796  0.411602  0.468553
RESTAURANT#PRICES          0.076923  0.142857  0.100000
LOCATION#GENERAL           0.326923  0.253731  0.285714
RESTAURANT#GENERAL         0.303226  0.406926  0.347505
SERVICE#GENERAL            0.713568  0.622807  0.665105
AMBIENCE#GENERAL           0.569892  0.517073  0.542199
DRINKS#PRICES              0.600000  0.204545  0.305085
RESTAURANT#MISCELLANEOUS   0.272727  0.051282  0.086331
DRINKS#STYLE&OPTIONS       0.115385  0.300000  0.166667
DRINKS#QUALITY             0.344828  0.256410  0.294118
          Precision    Recall        F1
positive   0.635936  0.698016  0.665532
negative   0.367089  0.125541  0.187097
neutral    0.366890  0.236994  0.287972
F1: 0.5796585003711953


In [ ]:
val_data = preprocess_txt('/content/VLSP2018-SA-train-dev-test/3-VLSP2018-SA-Restaurant-test (8-3-2018).txt')
nTP, nTP_asp, npred, ntruth = 0, 0, 0, 0 
asp_f1 = {}
sent_f1 = {}
for pdata in val_data:
    pdata = (process_raw_dataset(pdata[0]), pdata[1])
    pred = predict(pdata[0], model, vectorizer, asp_lst)
    truth = pdata[1]

    nTP = nTP + check_pred(pred, truth)
    nTP_asp = nTP_asp + check_pred_asp(pred, truth)
    npred = npred + len(pred)
    ntruth = ntruth + len(truth)
    ints = set(pred) & set(truth)
    for m in ints:
        if m[0] not in asp_f1.keys():
            asp_f1[m[0]] = (0, 0, 0)
        if m[1] not in sent_f1.keys():
            sent_f1[m[1]] = (0, 0, 0)
        asp_f1[m[0]] = (asp_f1[m[0]][0] + 1, asp_f1[m[0]][1], asp_f1[m[0]][2])
        sent_f1[m[1]] = (sent_f1[m[1]][0] + 1, sent_f1[m[1]][1], sent_f1[m[1]][2])
    for m in set(pred) - ints:
        if m[0] not in asp_f1.keys():
            asp_f1[m[0]] = (0, 0, 0)
        if m[1] not in sent_f1.keys():
            sent_f1[m[1]] = (0, 0, 0)
        asp_f1[m[0]] = ((asp_f1[m[0]])[0], (asp_f1[m[0]])[1] + 1, (asp_f1[m[0]])[2])
        sent_f1[m[1]] = (sent_f1[m[1]][0], sent_f1[m[1]][1] + 1, sent_f1[m[1]][2])
    for m in set(truth) - ints:
        if m[0] not in asp_f1.keys():
            asp_f1[m[0]] = (0, 0, 0)
        if m[1] not in sent_f1.keys():
            sent_f1[m[1]] = (0, 0, 0)
        asp_f1[m[0]] = (asp_f1[m[0]][0], asp_f1[m[0]][1], asp_f1[m[0]][2] + 1)
        sent_f1[m[1]] = (sent_f1[m[1]][0], sent_f1[m[1]][1], sent_f1[m[1]][2] + 1)
print('TẬP TEST')
report = pd.DataFrame.from_dict(asp_f1, orient='index', columns=['TP', 'FP', 'FN'])
report['Precision'] = report['TP'] / (report['TP'] + report['FP'])
report['Recall'] = report['TP'] / (report['TP'] + report['FN'])
report['F1'] = 2 * (report['Precision'] * report['Recall']) / (report['Precision'] + report['Recall'])
report.drop(['TP', 'FN', 'FP'], axis=1, inplace=True)
print(report)
report = pd.DataFrame.from_dict(sent_f1, orient='index', columns=['TP', 'FP', 'FN'])
report['Precision'] = report['TP'] / (report['TP'] + report['FP'])
report['Recall'] = report['TP'] / (report['TP'] + report['FN'])
report['F1'] = 2 * (report['Precision'] * report['Recall']) / (report['Precision'] + report['Recall'])
report.drop(['TP', 'FN', 'FP'], axis=1, inplace=True)
print(report)
print('F1:', F1(nTP, npred, ntruth))

TẬP TEST
                          Precision    Recall        F1
RESTAURANT#GENERAL         0.533088  0.650224  0.585859
DRINKS#QUALITY             0.666667  0.366197  0.472727
AMBIENCE#GENERAL           0.608108  0.529412  0.566038
FOOD#STYLE&OPTIONS         0.727735  0.709677  0.718593
RESTAURANT#PRICES          0.239437  0.232877  0.236111
FOOD#QUALITY               0.816239  0.835886  0.825946
DRINKS#PRICES              0.291667  0.092105  0.140000
DRINKS#STYLE&OPTIONS       0.459459  0.369565  0.409639
SERVICE#GENERAL            0.459459  0.388571  0.421053
FOOD#PRICES                0.418831  0.389728  0.403756
RESTAURANT#MISCELLANEOUS   0.181818  0.030769  0.052632
LOCATION#GENERAL           0.419355  0.290503  0.343234
          Precision    Recall        F1
positive   0.643468  0.675854  0.659264
neutral    0.404605  0.208122  0.274860
negative   0.239437  0.106918  0.147826
F1: 0.5577303716736309
